In [4]:
from sklearn.ensemble import GradientBoostingClassifier
import warnings
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.model_selection import KFold
import pandas as pd
import os
import numpy as np
import support 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def under_sampling(x):
    import warnings
    import pandas as pd
    import os
    warnings.simplefilter('ignore', pd.core.common.SettingWithCopyWarning)
    df = x
    x_majority=df.loc[df.iloc[:,-1]==0]
    x_minority=df.loc[df.iloc[:,-1]==1]
    from sklearn.cluster import KMeans
    km = KMeans(random_state=1, n_clusters=10)
    km.fit(x_majority.iloc[:, :-1])
    x_majority['Cluster'] = km.predict(x_majority.iloc[:, :-1])
    ratio = x_majority['Cluster'].value_counts() / x_majority.shape[0]
    n_sample_ary = (ratio * x_minority.shape[0]).sort_index().astype('int64')
    df_Xs=[]
    for i, n_sample in enumerate(n_sample_ary):
        #重複を許すか許さないかで変わるのか？
        df_Xs.append(x_majority[x_majority['Cluster']==i].sample(n=n_sample, replace=True))

    df_Xs.append(x_minority)
    x=pd.concat(df_Xs, sort=True)
    x=x.drop('Cluster', axis=1)
    
    return x

def calc_avg(x, y):
    import pandas as pd
    import os
    df_x=pd.DataFrame(x)
    df_y=pd.DataFrame(y)
    df_x_raw=df_x[(df_x.shape[1]-1)/2]
    raw_df=pd.concat([df_x_raw, df_y], axis=1)
    raw_df.columns=range(raw_df.shape[1])
    df_ex=raw_df.loc[raw_df[1]==0]
    N=len(df_ex)
    Leq=10*np.log10(np.sum(np.power(10, df_ex[0]/10)))-10*np.log10(N)
    return Leq

def calc_avg2(x, y):
    import pandas as pd
    import os
    df_x=pd.DataFrame(x)
    df_y=pd.DataFrame(y)
    df_x_raw=df_x[(df_x.shape[1]-1)/2]
    raw_df=pd.concat([df_x_raw, df_y], axis=1)
    raw_df.columns=range(raw_df.shape[1])
    df_ex=raw_df
    N=len(df_ex)
    Leq=10*np.log10(np.sum(np.power(10, df_ex[0]/10)))-10*np.log10(N)
    return Leq

df = pd.read_csv( os.path.join('dataset', 'learning_data_431975.csv'), header=None  )
x = df[ df.columns[ :-1 ] ]
y=  df[ df.columns[ -1 ] ]
kf = KFold( n_splits=3, random_state=1, shuffle=True )
pr=[]
re=[]
f1 = []
ac = []
y_Leq=[]
pred_Leq=[]
no_Leq=[]
n = []

for train_index, test_index in kf.split( x ):
    x_train, x_test = x.loc[train_index, :].values, x.loc[test_index, :].values
    y_train, y_test = y.loc[train_index].values, y.loc[test_index].values

    # 学習データだけダウンサンプリング
    y_data=y_train[:, np.newaxis]
    data=np.concatenate([x_train, y_data], 1)
    train_df=pd.DataFrame(data)
    train_us=under_sampling(train_df)
    index = train_us.index.tolist()
    x_train=x_train[index]
    y_train=y_train[index]
   
    gbdt = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.01, max_depth=4,random_state=0)
    gbdt.fit(x_train, y_train)
    z = gbdt.predict(x_test)

    pr.append( precision_score( y_test, z ) )
    re.append( recall_score( y_test, z) )
    f1.append( f1_score( y_test, z ) )
    ac.append( accuracy_score( y_test, z ) )
    n.append( len( x_test ) / len( x ) )

    #等価騒音レベル計算
    no_Leq.append(calc_avg2(x_test, y_test))
    y_Leq.append(calc_avg(x_test, y_test))
    pred_Leq.append(calc_avg(x_test, z))

    
print( 'CV Score:' )
print( '  Precision Score = %f'%( np.average( pr, weights=n ) ) )
print( '  Recall Score = %f'%( np.average( re, weights=n ) ) )
print( '  F1 Score = %f'%( np.average( f1, weights=n ) ) )
print( '  Accuracy Score = %f'%( np.average( ac, weights=n ) ) )
print( '  no_Leq = %f'%( np.average( no_Leq, weights=n ) ) )
print( '  y_Leq = %f'%( np.average( y_Leq, weights=n ) ) )
print( '  pred_Leq = %f'%( np.average( pred_Leq, weights=n ) ) )

CV Score:
  Precision Score = 0.021216
  Recall Score = 0.804136
  F1 Score = 0.041340
  Accuracy Score = 0.582925
  no_Leq = 55.975535
  y_Leq = 55.937235
  pred_Leq = 55.270739
